In [120]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import requests
import os

In [122]:
os. chdir('C:\\Users\\32493\\Google Drive\\ku leuven\\data analysis\\MDA\\CSV FILES')

In [150]:
url = 'https://raw.githubusercontent.com/Arthur-Vansimpsen/MDA/refs/heads/main/dataset3_fielddummies.csv?token=GHSAT0AAAAAADBGGFA3GXT5OUKOR7D3BIQM2BK7SOA'
Dataset = pd.read_csv(url)
print(Dataset.tail(n=10))
print(Dataset.info())

       Unnamed: 0  projectID  agricultural sciences  \
15331       15331  190197288                    0.0   
15332       15332  190197515                    0.0   
15333       15333  190198093                    0.0   
15334       15334  190198268                    0.0   
15335       15335  190198323                    0.0   
15336       15336  190198819                    0.0   
15337       15337  190199097                    0.0   
15338       15338  190199127                    0.0   
15339       15339  190199469                    0.0   
15340       15340  190199874                    0.0   

       engineering and technology  humanities  medical and health sciences  \
15331                         3.0         0.0                          0.0   
15332                         0.0         0.0                          2.0   
15333                         0.0         0.0                          0.0   
15334                         0.0         0.0                          2.0   
1533

Set original missing values in proportion_cost again to Nan


In [153]:
Dataset.loc[Dataset['mask'] == True, 'proportion_cost'] =  np.nan
Dataset.isna().sum()

Unnamed: 0                        0
projectID                         0
agricultural sciences          1799
engineering and technology     1799
humanities                     1799
medical and health sciences    1799
natural sciences               1799
social sciences                1799
startDate                         0
endDate                           0
totalCost                         0
ecMaxContribution                 0
fundingScheme                     1
duration                          0
month                             0
country                           0
role                              0
activityType                      1
city                              0
contributors                      0
proportion_cost                6564
mask                              0
logCost                           0
dtype: int64

In [155]:
for col in ['fundingScheme', 'country', 'activityType', 'social sciences']:
    print(f"\nUnique values in {col}:")
    print(Dataset[col].unique())


Unique values in fundingScheme:
['HORIZON-ERC' 'HORIZON-AG' 'ERC' 'HORIZON-CSA' 'HORIZON-RIA'
 'HORIZON-EIC' 'EURATOM-COFUND' 'MSCA-PF' 'CSA' 'HORIZON-COFUND'
 'HORIZON-TMA-MSCA-PF-EF' 'HORIZON-AG-LS' 'HORIZON-AG-UN' 'HORIZON-IA'
 'HORIZON-ERC-POC' 'EIC' 'HORIZON-TMA-MSCA-PF-GF' 'EURATOM-CSA'
 'EURATOM-RIA' 'ERC-POC' 'EURATOM-IA' 'HORIZON-TMA-MSCA-DN-ID'
 'HORIZON-TMA-MSCA-DN' 'HORIZON-ERC-SYG' 'ERC-SYG'
 'HORIZON-TMA-MSCA-DN-JD' 'RIA' 'HORIZON-TMA-MSCA-SE'
 'HORIZON-TMA-MSCA-Cofund-P' 'HORIZON-TMA-MSCA-Cofund-D' 'IA'
 'HORIZON-PCP' 'HORIZON-JU-RIA' 'HORIZON-JU-IA' 'HORIZON-JU-CSA'
 'HORIZON-EIT-KIC' nan 'HORIZON-EIC-ACC-BF' 'HORIZON-EIC-ACC' 'EIC-ACC']

Unique values in country:
['ES' 'PL' 'DE' 'FR' 'FI' 'IL' 'NL' 'NO' 'IT' 'SE' 'PT' 'TR' 'CZ' 'AT'
 'DK' 'BE' 'IE' 'EL' 'IS' 'LU' 'SI' 'CH' 'RO' 'HU' 'EE' 'CY' 'UK' 'AL'
 'RS' 'ME' 'SK' 'LT' 'BA' 'FO' 'MD' 'MT' 'BG' 'HR' 'LV' 'UA' 'TN' 'GE'
 'ZA' 'AM' 'KE' 'GH' 'NG' 'MK' 'AU']

Unique values in activityType:
['REC' 'HES' 'OTH' 'PRC' 'PU

Drop the missing values in activitytype and fundingscheme (only 2).

In [158]:
Dataset = Dataset.dropna(subset=['activityType'])
Dataset = Dataset.dropna(subset=['fundingScheme'])
Dataset.drop(columns = ['mask', 'logCost', 'startDate', 'endDate', 'totalCost', 'Unnamed: 0', 'projectID', 'city','role'], inplace=True)
print(Dataset)
print(Dataset.isna().sum())

       agricultural sciences  engineering and technology  humanities  \
0                        0.0                         0.0         0.0   
1                        0.0                         0.0         2.0   
2                        1.0                         0.0         0.0   
3                        0.0                         0.0         0.0   
4                        0.0                         0.0         0.0   
...                      ...                         ...         ...   
15336                    0.0                         4.0         0.0   
15337                    0.0                         1.0         0.0   
15338                    0.0                         0.0         0.0   
15339                    0.0                         1.0         0.0   
15340                    0.0                         0.0         0.0   

       medical and health sciences  natural sciences  social sciences  \
0                              0.0               2.0          

split in training and test set.

In [161]:
x = Dataset.drop('ecMaxContribution', axis=1)
y = Dataset['ecMaxContribution']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

New column 'split' that indicates if observation was a train or test

In [164]:
Dataset['split'] = 'test'
Dataset.loc[x_train.index, 'split'] = 'train'

In [142]:
Dataset.to_csv('Dataset_for_model.csv', index=False)